In [ ]:
# --- Step 1: Install Ultralytics ---
%pip install ultralytics > /dev/null 2>&1
print("--- Ultralytics installed ---")

import os
import shutil
import yaml
from ultralytics import YOLO

# --- Step 2: Create the CORRECT data.yaml --- MAKE SURE PATHS ARE CORRECT
true_names_path = '/kaggle/input/military-aircraft-detection-dataset-yolo-format/military/aircraft_names.yaml'
with open(true_names_path, 'r') as file:
    correct_class_names = yaml.safe_load(file)['names']
data_yaml_content = {
    'train': '/kaggle/input/military-aircraft-detection-dataset-yolo-format/military/images/aircraft_train',
    'val': '/kaggle/input/military-aircraft-detection-dataset-yolo-format/military/images/aircraft_val',
    'nc': len(correct_class_names),
    'names': correct_class_names
}
with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml_content, outfile, default_flow_style=False)
print("--- CORRECT data.yaml created ---")


# --- Step 3: Load the BEST Hyperparameters ---
hyp_path = '/kaggle/input/hyperparameters/best_hyperparameters.yaml' # from tuning session
with open(hyp_path, 'r') as f:
    best_params = yaml.safe_load(f)
print(f"--- Best hyperparameters loaded from {hyp_path} ---")


# --- Step 4: Define the Protection Callback ---
persistent_model_dir = "/kaggle/working/persistent_models/"
os.makedirs(persistent_model_dir, exist_ok=True)
def save_checkpoint_callback(trainer):
    last_pt_path, best_pt_path = trainer.last, trainer.best
    if os.path.exists(last_pt_path): shutil.copy(last_pt_path, os.path.join(persistent_model_dir, "last.pt"))
    if os.path.exists(best_pt_path): shutil.copy(best_pt_path, os.path.join(persistent_model_dir, "best.pt"))
print("--- Checkpoint callback defined ---")


# --- Step 5: Initialize Model and Attach Callback ---
model = YOLO('yolov8m.pt') # fresh model
model.add_callback("on_model_save", save_checkpoint_callback)


# --- Step 6: Start a Training Run ---
print("\n--- Starting training ---")
results = model.train(
    data='data.yaml',
    epochs=35, # make sure it doesn't take more than 12 hours
    imgsz=800,
    patience=5,
    project='aircraft_detection_final',
    name='correct_run_yolov8m_optimized',
    batch=-1,
    workers=4, # set accordingly with available CPU cores
    optimizer='SGD',
    **best_params
)

print("\n--- Correct training complete! ---")